In [35]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [36]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [37]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [38]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [39]:
# reflect an existing database into a new model
Base=automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [40]:
# View all of the classes that automap found
inspector = inspect(engine)
print(inspector.get_table_names())

['measurement', 'station']


In [41]:
# Save references to each table
Station=Base.classes.station
Measurement=Base.classes.measurement

In [42]:
# Create our session (link) from Python to the DB
session = Session(bind=engine)

# Exploratory Precipitation Analysis

In [58]:
# Find the most recent date in the data set.
sel=[Measurement.station,
    Measurement.date,
    Measurement.prcp,
    Measurement.tobs]
recent=session.query(*sel).order_by(Measurement.date.desc()).first()
recent

('USC00519397', '2017-08-23', 0.0, 81.0)

In [44]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.


# Perform a query to retrieve the data and precipitation scores


# Save the query results as a Pandas DataFrame and set the index to the date column


# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data




In [45]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [46]:
# Design a query to calculate the total number stations in the dataset


In [47]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [48]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [49]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [50]:
# Close Session
session.close()